#### You can refer to the [commands](./commands.md) file for the list of commands utilized in this demo.

In [1]:
!$SPARK_HOME/sbin/start-master.sh
!$SPARK_HOME/sbin/start-worker.sh spark://YDEARYZEN.:7077

starting org.apache.spark.deploy.master.Master, logging to /home/shanks/hadoop/spark-3.5.1/logs/spark-shanks-org.apache.spark.deploy.master.Master-1-YDEARYZEN.out
starting org.apache.spark.deploy.worker.Worker, logging to /home/shanks/hadoop/spark-3.5.1/logs/spark-shanks-org.apache.spark.deploy.worker.Worker-1-YDEARYZEN.out


In [5]:
import pandas as pd
import os
import json
from glob import glob
import pathlib
import uuid

### Check your data folder to ensure availability of files to process

In [3]:
os.listdir(f"/home/{os.environ.get('USER')}/setu/examples/sample_data/english")

['02448f577d0bcc8f438dd06fafada4d33711b361.json',
 '05854b4f9d0751b81403bf3436ed03559fa9c3ce.json',
 '07364b2b241fd42bee97cf7031f705467a29be7b.json',
 '072fe8ae637c266d6410b9d1d0657ecb3cc6c468.json',
 '066fa21773ec16f318d89478bbdc2608548f8d5f.json',
 '0606c1eb13b654fd61cb1e9be1c80a9fd4f01889.json',
 '0064da3e58e33153f4b415acbf1aa35c526c9657.json',
 '017964151c474e9f261922ba78f4fe8f29ecdb88.json',
 '0671ad9983db465c43c89344d419a81d9ca56ba4.json',
 '0318dc51a06f31e72074cf359b30f666ecbd5152.json',
 '053e749677b01684e2f20c31e3ba23426588519d.json',
 '037590d2a82f5efa532fc80519e9e022e901783e.json',
 '053e16c0b61aba25235662fc2717126b44a44702.json',
 '06951af315ca9b94e2a016bc4d6780c8a22d0e2a.json',
 '06800df8ebb9232796d5f0a9f7897bd3a4fee56a.json',
 '07305d21afe5335bf80f2a4ef5df474101e67865.json',
 '03070278f98cbb9bf9f94529f5c3d0d0666cd7a4.json',
 '0371f3ab284b050ce35fa156798e42408e35af24.json',
 '0200bd2bf9e071216c4e46802ef9bceef5ead1fd.json',
 '05230ba4e1461beb11c11771a45e9b7b508d9b28.json',


### Run the JSON2Parquet Stage to convert your data jsons into parquets

In [4]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    JSON2ParquetStage \
    --json_glob_path "/home/$USER/setu/examples/sample_data/english/*.json" \
    --language english \
    --j2p_samples_per_partition 1500 \
    --j2p_parquet_output_path /home/$USER/setu/examples/output/j2p_output

24/03/28 02:30:33 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 02:30:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 02:30:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

### Check the JSON2Parquet Stage Output

In [5]:
json_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/j2p_output"

In [6]:
parquets = glob(f"{json_output_path}/*.parquet")

In [7]:
df = pd.read_parquet(parquets[0])

In [8]:
df.head()

,doc_id,url,source,language,text,timestamp
0,dcff18db-2742-4811-a826-9aa4175cc479,https://www.91mobiles.com/marathi/iqoo-neo-7-p...,91mobiles,english,"<html dir=""ltr"" lang=""en-US""> \r\n<head>\r\n ...",10/07/23 22:20
1,667951e8-fddf-477c-af7d-7e625b3ae12f,https://www.91mobiles.com/hub/tag/nothing-phon...,91mobiles,english,"<html lang=""en-US""> <head><meta charset=""UTF-8...",10/07/23 22:05
2,8056b2ea-db81-4faa-99e9-2a57527b24c2,https://www.91mobiles.com/compare/Apple/iPhone...,91mobiles,english,"<html lang=""en"">\n\t<head>\n\t\t \n ...",10/07/23 20:47
3,601ce925-c9ca-4e67-9591-d0fac21e5c83,https://www.flipkart.com/iqoo-9t-5g-legend-128...,91mobiles,english,"<html lang=""en""><head><link href=""https://rukm...",10/07/23 11:58
4,04fd5bec-3f24-4170-8f00-2ce74065c5a9,https://www.91mobiles.com/hub/tag/rpgs-for-pc/,91mobiles,english,"<html lang=""en-US""> <head><meta charset=""UTF-8...",10/07/23 21:59


### Run the Text Extraction State to extract text from the previous stage output

In [9]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    ExtractTextStage \
    --te_parquets_path "/home/$USER/setu/examples/output/j2p_output/*.parquet" \
    --te_samples_per_partition 1500 \
    --te_run_mode data \
    --te_output_path "/home/$USER/setu/examples/output/te_output"


24/03/28 02:31:00 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 02:31:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 02:31:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

### Check the TextExtract Stage Output

In [10]:
te_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/te_output"

In [11]:
parquets = glob(f"{te_output_path}/*.parquet")

In [12]:
parquets

['/home/shanks/setu/examples/output/te_output/part-00000-83cf4588-1d65-4ce6-9d04-edf8d244f0a8-c000.snappy.parquet']

In [13]:
df = pd.read_parquet(parquets[0])

In [14]:
df.head()

,doc_id,url,source,timestamp,language,successful_extraction,title,description,text,comments,...,categories,tags,fingerprint,id,license,body,commentsbody,raw_text,image,pagetype
0,0139bd54-6abb-4e6b-8f67-2b1626e6e90b,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,10/07/23 21:51,english,True,Samsung Galaxy S20 FE 5G vs vivo X80 5G,None,"Rs. 79,999\nRs. 66,999\nNA\nNA\n|\nSummary\n|P...",,...,[],[],None,None,None,None,None,None,None,None
1,01f17a1c-b4f8-4aa0-a974-503ee7b21b61,https://www.91mobiles.com/tablets/compare/Asus...,91mobiles,10/07/23 23:10,english,True,Asus Fonepad 8 8GB vs Asus Fonepad 8 16GB,None,|\nSummary\n|General\n|\nDual SIM\nN/A\n|Displ...,,...,[],[],None,None,None,None,None,None,None,None
2,0211005f-a277-42af-88f5-00458bc2a40b,https://www.91mobiles.com/sony-vpceb42eg-core-...,91mobiles,10/07/23 23:37,english,True,Sony VAIO E VPCEB42EG Laptop (Core i3 1st Gen/...,None,1. Trusted Brand\n2. A leader in various elect...,,...,[],[],None,None,None,None,None,None,None,None
3,02389e48-71aa-41b1-ab72-5ba763309178,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,10/07/23 22:43,english,True,Samsung Galaxy A5 vs Samsung Galaxy A7,None,"Rs. 23,499\nNA\n|\nSummary\n|Performance\n|\nS...",,...,[],[],None,None,None,None,None,None,None,None
4,037f914c-c4dc-42ac-8444-e2b1821ffe0d,https://www.91mobiles.com/recharge-plans/airte...,91mobiles,10/07/23 21:01,english,True,Airtel Haryana 1 GB Per Day Plans,None,|Pack / Plan\n|Validity\n|Price (Rs.)\n|1 GB /...,,...,[],[],None,None,None,None,None,None,None,None


### Run DocCleanStage on the Text Extraction Stage output

In [15]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    DocCleanStage \
    --doc_df_parquets_path "/home/$USER/setu/examples/output/te_output/*.parquet" \
    --is_doc_df_path_batched False \
    --doc_clean_additional_cols_to_use "url,source,language" \
    --use_symbol_filter True \
    --doc_clean_samples_per_partition 1500 \
    --doc_clean_verbose False \
    --doc_clean_run_mode data \
    --symbol_filter_output_path "/home/$USER/setu/examples/output/symbol_filter/" \
    --cleaned_doc_output_path "/home/$USER/setu/examples/output/cleaned_docs/" 

24/03/28 02:36:45 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 02:36:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 02:36:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

### Check DocClean Stage Output

In [16]:
cd_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/cleaned_docs"

In [17]:
parquets = glob(f"{cd_output_path}/*.parquet")

In [18]:
df = pd.read_parquet(parquets[0])

In [19]:
df.head()

,doc_id,text,url,source,language,code_spans,uncleaned_chars_count,uncleaned_words_count,uncleaned_bytes,symbol_ratio,invalid_char_count,uncleaned_text
0,00e61c2d-ae0d-4c35-be0b-9f86c39a4ff6,2. Samsung is one of the most popular smartpho...,https://www.91mobiles.com/samsung-2-360-13-3-c...,91mobiles,english,None,2387,254,2387,0.088395,211,1. Trusted Brand\n2. Samsung is one of the mos...
1,0107bd12-1073-484f-96b1-757eaea0dd9c,Its a decent tab at a big billion day price of...,https://www.91mobiles.com/Lenovo-Tab+4+8+16GB+...,91mobiles,english,None,1002,183,1002,0.031936,32,Its a decent tab at a big billion day price of...
2,0139bd54-6abb-4e6b-8f67-2b1626e6e90b,"6.5"" (16.51 cm)\n6.78"" (17.22 cm)\n1 TB (Expan...",https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,english,None,7270,815,7275,0.136176,990,"Rs. 79,999\nRs. 66,999\nNA\nNA\n|\nSummary\n|P..."
3,01f17a1c-b4f8-4aa0-a974-503ee7b21b61,"8.0"" (20.32 cm)\n64 GB (Expandable)\n64 GB (Ex...",https://www.91mobiles.com/tablets/compare/Asus...,91mobiles,english,None,3650,341,3658,0.131233,479,|\nSummary\n|General\n|\nDual SIM\nN/A\n|Displ...
4,02303483-e390-42e1-8ea8-79c545bf4b2f,The V series has been Vivo’s top mid-range off...,https://www.91mobiles.com/hub/vivo-v20-pro-unb...,91mobiles,english,None,3516,618,3528,0.027873,98,The V series has been Vivo’s top mid-range off...


### Run the LIDStage

In [2]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 3 \
    --executor-cores 2 \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    LIDStage \
    --lid_df_parquets_path "/home/$USER/setu/examples/output/cleaned_docs/*.parquet" \
    --is_lid_df_path_batched False \
    --lid_additional_cols "url,source,language" \
    --lid_samples_per_partition 1500 \
    --lid_verbose False \
    --lid_run_mode data \
    --doc_lid_output_path "/home/$USER/setu/examples/output/lid/"

24/03/28 03:23:12 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 03:23:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 03:23:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

### Check LIDStage Output 

In [6]:
lid_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/lid/doc_lang_partition=english"

In [7]:
parquets = glob(f"{lid_output_path}/*.parquet")

In [8]:
df = pd.read_parquet(parquets[0])

In [9]:
df.head()

,doc_id,url,source,language,text,doc_lang,doc_lang_iso,indiclid_code,indiclid_script,indiclid_lang,nllb_lang,cld3_lang,indiclid_logit,nllb_probability,cld3_probability
0,00e61c2d-ae0d-4c35-be0b-9f86c39a4ff6,https://www.91mobiles.com/samsung-2-360-13-3-c...,91mobiles,english,2. Samsung is one of the most popular smartpho...,english,eng,eng_Latn,latin,english,english,english,0.996126,0.629310,0.999099
1,0107bd12-1073-484f-96b1-757eaea0dd9c,https://www.91mobiles.com/Lenovo-Tab+4+8+16GB+...,91mobiles,english,Its a decent tab at a big billion day price of...,english,eng,eng_Latn,latin,english,english,english,1.000048,0.999845,0.999994
2,0139bd54-6abb-4e6b-8f67-2b1626e6e90b,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,english,"6.5"" (16.51 cm)\n6.78"" (17.22 cm)\n1 TB (Expan...",english,eng,eng_Latn,latin,english,english,english,0.830918,0.212418,0.998820
3,01f17a1c-b4f8-4aa0-a974-503ee7b21b61,https://www.91mobiles.com/tablets/compare/Asus...,91mobiles,english,"8.0"" (20.32 cm)\n64 GB (Expandable)\n64 GB (Ex...",english,eng,eng_Latn,latin,english,english,english,0.998657,0.492814,0.997142
4,02303483-e390-42e1-8ea8-79c545bf4b2f,https://www.91mobiles.com/hub/vivo-v20-pro-unb...,91mobiles,english,The V series has been Vivo’s top mid-range off...,english,eng,eng_Latn,latin,english,english,english,1.000044,0.999740,0.999989


### Run the Analysis Stages

In [10]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    AnalysisStage \
    --analysis_df_parquets_path "/home/$USER/setu/examples/output/lid/*/*.parquet" \
    --is_analysis_df_path_batched False \
    --analysis_additional_cols_to_use "url,source,language,doc_lang,doc_lang_iso" \
    --analysis_samples_per_partition 1500 \
    --analysis_verbose False \
    --analysis_run_mode stage \
    --line_stats_output_path "/home/$USER/setu/examples/output/line_stats/" \
    --doc_stats_output_path "/home/$USER/setu/examples/output/doc_stats/" \
    --analysis_output_path "/home/$USER/setu/examples/output/analysis/"

24/03/28 03:25:43 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 03:25:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 03:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

### View Analysis Output

In [11]:
al_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/doc_stats/doc_lang_partition=english"

In [12]:
parquets = glob(f"{al_output_path}/*.parquet")

In [13]:
parquets

['/home/shanks/setu/examples/output/doc_stats/doc_lang_partition=english/part-00000-b518f0f0-1da3-4601-875a-fe3d51d93975.c000.gz.parquet']

In [14]:
df = pd.read_parquet(parquets[0])

In [15]:
df.head()

,doc_id,bytes,words_count,char_count,lines_count,mean_line_length,min_line_length,max_line_length,nsfw_words_count,non_li_char_count,char_ngram_repetition_score,10_gram_characters_repetition_score,word_ngram_repetition_score,5_gram_words_repetition_score,doc_lang
0,8d82a88e-5f10-48a1-ad55-04ca38ff3f18,269,52,269,8,6.500000,1,21,0,17,"[(10_gram_characters_repetition_score, 0.11923...",0.119231,"[(5_gram_words_repetition_score, None)]",NaN,english
1,932c4813-5508-4e7a-bc74-41d986071681,266,31,266,5,6.200000,1,17,0,25,"[(10_gram_characters_repetition_score, 0.10894...",0.108949,"[(5_gram_words_repetition_score, 0.16216215491...",0.162162,english
2,d00750ab-59ed-4250-9b26-d7519bcd84f0,323,45,323,5,9.000000,3,21,0,24,"[(10_gram_characters_repetition_score, 0.09554...",0.095541,"[(5_gram_words_repetition_score, 0.32258063554...",0.322581,english
3,ea443fc8-0ca6-4f24-b549-2d870c1e584b,811,134,811,7,19.142857,1,24,0,2,"[(10_gram_characters_repetition_score, 0.06608...",0.066085,"[(5_gram_words_repetition_score, 0.60431653261...",0.604317,english
4,0e2c2e1f-37d0-49ea-b9a0-1d346658d319,1293,215,1293,24,8.958333,1,39,0,279,"[(10_gram_characters_repetition_score, 0.08099...",0.080997,"[(5_gram_words_repetition_score, 0.77752810716...",0.777528,english


# Run the Flagging and Filtering Stage

In [16]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    FlaggingAndFilteringStage \
    --doc_stats_parquets_path "/home/$USER/setu/examples/output/doc_stats/*/*.parquet" \
    --is_doc_stats_path_batched False \
    --fnf_samples_per_partition 1500 \
    --fnf_verbose False \
    --fnf_run_mode stage \
    --save_nsfw_data True \
    --nsfw_output_path "/home/$USER/setu/examples/output/nsfw/" \
    --filtered_doc_stats_output_path "/home/$USER/setu/examples/output/filtered_doc_stats/"

24/03/28 03:26:38 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 03:26:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 03:26:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

### View Filtered Document Statistics

In [17]:
fd_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/filtered_doc_stats/"

In [18]:
parquets = glob(f"{fd_output_path}/*.parquet")

In [19]:
parquets

['/home/shanks/setu/examples/output/filtered_doc_stats/part-00000-289e2ce9-f153-450e-9b3c-8413ed34fe0a-c000.gz.parquet']

In [20]:
df.head()

,doc_id,bytes,words_count,char_count,lines_count,mean_line_length,min_line_length,max_line_length,nsfw_words_count,non_li_char_count,char_ngram_repetition_score,10_gram_characters_repetition_score,word_ngram_repetition_score,5_gram_words_repetition_score,doc_lang
0,8d82a88e-5f10-48a1-ad55-04ca38ff3f18,269,52,269,8,6.500000,1,21,0,17,"[(10_gram_characters_repetition_score, 0.11923...",0.119231,"[(5_gram_words_repetition_score, None)]",NaN,english
1,932c4813-5508-4e7a-bc74-41d986071681,266,31,266,5,6.200000,1,17,0,25,"[(10_gram_characters_repetition_score, 0.10894...",0.108949,"[(5_gram_words_repetition_score, 0.16216215491...",0.162162,english
2,d00750ab-59ed-4250-9b26-d7519bcd84f0,323,45,323,5,9.000000,3,21,0,24,"[(10_gram_characters_repetition_score, 0.09554...",0.095541,"[(5_gram_words_repetition_score, 0.32258063554...",0.322581,english
3,ea443fc8-0ca6-4f24-b549-2d870c1e584b,811,134,811,7,19.142857,1,24,0,2,"[(10_gram_characters_repetition_score, 0.06608...",0.066085,"[(5_gram_words_repetition_score, 0.60431653261...",0.604317,english
4,0e2c2e1f-37d0-49ea-b9a0-1d346658d319,1293,215,1293,24,8.958333,1,39,0,279,"[(10_gram_characters_repetition_score, 0.08099...",0.080997,"[(5_gram_words_repetition_score, 0.77752810716...",0.777528,english


### Run Document Removal Stage

In [21]:
!SETU_DIR=/home/$USER/setu SETU_TMP_DIR=/home/$USER/tmp/ FILTER_DATA_ROOT=/home/$USER/setu/setu/data \
    spark-submit \
    --master spark://YDEARYZEN.:7077 \
    --deploy-mode client \
    --driver-java-options -Djava.io.tmpdir=/home/$USER/tmp/ \
    --conf "spark.driver.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf "spark.executor.extraJavaOptions=-Djava.io.tmpdir=/home/$USER/tmp/" \
    --conf spark.worker.dir="/home/$USER/tmp/" \
    --conf spark.local.dir="/home/$USER/tmp/" \
    --num-executors 4 \
    --executor-cores 2 \
    --executor-memory 3G \
    --driver-memory 6G \
    --archives "/home/$USER/setu/dataproc/envs/setu.zip" \
    --conf 'spark.executorEnv.PYTHONPATH=setu.zip' \
    --conf 'spark.executorEnv.FILTER_DATA_ROOT=setu.zip/data' \
    /home/$USER/setu/setu/run.py \
    --config /home/$USER/setu/configs/crawls/spark_english_config.json \
    --mode crawl \
    --run_local True \
    DocumentRemovalStage \
    --analysis_out_path "/home/$USER/setu/examples/output/analysis/*/*.parquet" \
    --doc_stats_path "/home/$USER/setu/examples/output/doc_stats/*/*.parquet" \
    --doc_removal_join_col "doc_id" \
    --doc_removal_samples_per_partition 1500 \
    --doc_removal_verbose False \
    --doc_removal_run_mode stage \
    --filtered_docs_path "/home/$USER/setu/examples/output/filtered_doc_stats/"

24/03/28 03:26:59 WARN Utils: Your hostname, YDEARYZEN resolves to a loopback address: 127.0.1.1; using 172.31.108.238 instead (on interface eth0)
24/03/28 03:26:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 03:27:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
shanks
------------------------------------------------ Setting Environment Variables --------------------------------------------------
------------------------------------------------ END --------------------------------------------------
------------------------------------------------ Environment Variables --------------------------------------------------
PATH: /home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/shanks/hadoop/spark-3.5.1/bin:/home/shanks/miniconda3/envs/setu_env/bin:/home/shanks/miniconda3/condabin:/usr/local/s

## View Final Doc Stats output

In [22]:
fd_output_path = f"/home/{os.environ.get('USER')}/setu/examples/output/filtered_doc_stats/"

In [23]:
parquets = glob(f"{fd_output_path}/*.parquet")

In [24]:
df = pd.read_parquet(parquets[0])

In [25]:
df.head()

,doc_id,url,source,language,doc_lang,doc_lang_iso,text,bytes,words_count,char_count,lines_count,mean_line_length,min_line_length,max_line_length,nsfw_words_count,non_li_char_count,char_ngram_repetition_score,10_gram_characters_repetition_score,word_ngram_repetition_score,5_gram_words_repetition_score
0,0034bd01-5b57-4521-9781-5cef067e8b4f,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,english,english,eng,"6.7"" (17.02 cm)\n6.9"" (17.53 cm)\nAugust 16, 2...",2374,421,2368,37,11.378378,1,69,0,560,"[(10_gram_characters_repetition_score, 0.12038...",0.120390,"[(5_gram_words_repetition_score, 0.83455884456...",0.834559
1,006f8fde-fe25-4db6-b0a9-9ed956094e91,https://www.91mobiles.com/tv/compare/sony/brav...,91mobiles,english,english,eng,Sony BRAVIA KLV-40R482B 40 inch (101 cm) LED F...,637,82,637,9,9.111111,1,19,0,60,"[(10_gram_characters_repetition_score, 0.10350...",0.103503,"[(5_gram_words_repetition_score, 0.51666665077...",0.516667
2,00e61c2d-ae0d-4c35-be0b-9f86c39a4ff6,https://www.91mobiles.com/samsung-2-360-13-3-c...,91mobiles,english,english,eng,2. Samsung is one of the most popular smartpho...,843,149,843,24,6.208333,1,19,0,98,"[(10_gram_characters_repetition_score, 0.09352...",0.093525,"[(5_gram_words_repetition_score, 0.20289854705...",0.202899
3,0107bd12-1073-484f-96b1-757eaea0dd9c,https://www.91mobiles.com/Lenovo-Tab+4+8+16GB+...,91mobiles,english,english,eng,Its a decent tab at a big billion day price of...,1003,204,1003,22,9.272727,1,27,0,13,"[(10_gram_characters_repetition_score, 0.04024...",0.040241,"[(5_gram_words_repetition_score, None)]",NaN
4,0139bd54-6abb-4e6b-8f67-2b1626e6e90b,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,english,english,eng,"6.5"" (16.51 cm)\n6.78"" (17.22 cm)\n1 TB (Expan...",2090,366,2085,35,10.457143,1,61,0,473,"[(10_gram_characters_repetition_score, 0.10211...",0.102119,"[(5_gram_words_repetition_score, 0.70792770385...",0.707928


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   doc_id                               1110 non-null   object 
 1   url                                  1110 non-null   object 
 2   source                               1110 non-null   object 
 3   language                             1110 non-null   object 
 4   doc_lang                             1110 non-null   object 
 5   doc_lang_iso                         1101 non-null   object 
 6   text                                 1110 non-null   object 
 7   bytes                                1110 non-null   int64  
 8   words_count                          1110 non-null   int64  
 9   char_count                           1110 non-null   int64  
 10  lines_count                          1110 non-null   int64  
 11  mean_line_length              

### Prepare for Deduplication

In [27]:
columns = df.columns
columns

Index(['doc_id', 'url', 'source', 'language', 'doc_lang', 'doc_lang_iso',
       'text', 'bytes', 'words_count', 'char_count', 'lines_count',
       'mean_line_length', 'min_line_length', 'max_line_length',
       'nsfw_words_count', 'non_li_char_count', 'char_ngram_repetition_score',
       '10_gram_characters_repetition_score', 'word_ngram_repetition_score',
       '5_gram_words_repetition_score'],
      dtype='object')

In [28]:
new_columns = ['doc_id', 'url', 'source', 'language', 'doc_lang', 'doc_lang_iso',
       'text', 'bytes', 'words_count', 'char_count', 'lines_count',
       'mean_line_length', 'min_line_length', 'max_line_length',
       'nsfw_words_count', 'non_li_char_count']

In [29]:
new_df = df[new_columns]

In [30]:
new_df.to_parquet("/home/shanks/setu/examples/output/prededup/prededup.parquet")

#### Run Minhash Deduplication on the Filtered Docs output

In [31]:
!python ../text-dedup/text_dedup/minhash.py \
    --path "parquet" \
    --name "sangraha-english" \
    --split "train" \
    --data_files "/home/shanks/setu/examples/output/prededup/*.parquet" \
    --cache_dir "/home/shanks/cache" \
    --output "/home/$USER/setu/examples/output/minhash/" \
    --column "text" \
    --batch_size 10000

Generating train split: 1110 examples [00:00, 97189.68 examples/s]
Fingerprinting... (num_proc=12): 100%|█| 1110/1110 [00:00<00:00, 3729.74 example
Clustering...: 100%|██████████████████████████| 25/25 [00:00<00:00, 3255.44it/s]
Finding clusters... (num_proc=12): 100%|█| 1110/1110 [00:00<00:00, 7988.37 examp
Filtering clusters... (num_proc=12): 100%|█| 1110/1110 [00:00<00:00, 9516.88 exa
Saving the dataset (1/1 shards): 100%|█| 918/918 [00:00<00:00, 66149.62 examples
[03/28/24 03:27:17] INFO     Loading                         :    ]8;id=784254;file:///home/shanks/setu/examples/../text-dedup/text_dedup/minhash.py\minhash.py]8;;\:]8;id=367333;file:///home/shanks/setu/examples/../text-dedup/text_dedup/minhash.py#302\302]8;;\
                             1.27s                                              
                    INFO     MinHashing                      :    ]8;id=982672;file:///home/shanks/setu/examples/../text-dedup/text_dedup/minhash.py\minhash.py]8;;\:]8;id=24

### Load and View your final dataset

In [32]:
from datasets import Dataset

ds = Dataset.from_file("/home/shanks/setu/examples/output/minhash/data-00000-of-00001.arrow")

/home/shanks/miniconda3/envs/setu_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
df = ds.to_pandas()

In [34]:
df.head()

,doc_id,url,source,language,doc_lang,doc_lang_iso,text,bytes,words_count,char_count,lines_count,mean_line_length,min_line_length,max_line_length,nsfw_words_count,non_li_char_count
0,0034bd01-5b57-4521-9781-5cef067e8b4f,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,english,english,eng,"6.7"" (17.02 cm)\n6.9"" (17.53 cm)\nAugust 16, 2...",2374,421,2368,37,11.378378,1,69,0,560
1,006f8fde-fe25-4db6-b0a9-9ed956094e91,https://www.91mobiles.com/tv/compare/sony/brav...,91mobiles,english,english,eng,Sony BRAVIA KLV-40R482B 40 inch (101 cm) LED F...,637,82,637,9,9.111111,1,19,0,60
2,00e61c2d-ae0d-4c35-be0b-9f86c39a4ff6,https://www.91mobiles.com/samsung-2-360-13-3-c...,91mobiles,english,english,eng,2. Samsung is one of the most popular smartpho...,843,149,843,24,6.208333,1,19,0,98
3,0107bd12-1073-484f-96b1-757eaea0dd9c,https://www.91mobiles.com/Lenovo-Tab+4+8+16GB+...,91mobiles,english,english,eng,Its a decent tab at a big billion day price of...,1003,204,1003,22,9.272727,1,27,0,13
4,0139bd54-6abb-4e6b-8f67-2b1626e6e90b,https://www.91mobiles.com/compare/Samsung/Gala...,91mobiles,english,english,eng,"6.5"" (16.51 cm)\n6.78"" (17.22 cm)\n1 TB (Expan...",2090,366,2085,35,10.457143,1,61,0,473
